In [11]:
import os
import openai
from dotenv import load_dotenv
load_dotenv()

openai.api_type = "azure"
openai.api_version = "2023-06-01-preview"

API_KEY = os.getenv("OPENAI_API_KEY","").strip()
assert API_KEY, "ERROR: Azure OpenAI Key is missing"
openai.api_key = API_KEY

RESOURCE_ENDPOINT = os.getenv("OPENAI_API_BASE","").strip()
assert RESOURCE_ENDPOINT, "ERROR: Azure OpenAI Endpoint is missing"
assert "openai.azure.com" in RESOURCE_ENDPOINT.lower(), "ERROR: Azure OpenAI Endpoint should be in the form: \n\n\t<your unique endpoint identifier>.openai.azure.com"
openai.api_base = RESOURCE_ENDPOINT

prompt="### Postgres SQL tables, with their properties:\n#\n# Employee(id, name, department_id)\n# Department(id, name, address)\n# Salary_Payments(id, employee_id, amount, date)\n#\n### A query to list the names of the departments which employed more than 10 employees in the last 3 months\n\ query: SELECT"
# prompt="""### Postgres SQL tables, with their properties:\n#\n# Employee(id, name, department_id)\n# Department(id, name, address)\n# Salary_Payments(id, employee_id, amount, date)\n#\n###
# 최근 3 개월간 직원을 10명 이상 뽑은 부서의 이름을 수집하는 쿼리  \n\ query: SELECT"""

"""한글 결과
Department.name FROM Department
INNER JOIN Employee ON Department.id = Employee.department_id
INNER JOIN Salary_Payments ON Employee.id = Salary_Payments.employee_id
WHERE Salary_Payments.date >= NOW() - INTERVAL '3 months'
GROUP BY Department.id, Department.name
HAVING COUNT(DISTINCT Employee.id) >= 10
"""
print(prompt+"\n")

response = openai.ChatCompletion.create(
  messages=[
    {"role": "user", "content": prompt},
  ],
  engine=os.getenv('DEPLOYMENT_NAME'),
  temperature=0,
  max_tokens=150,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0,
  stop=["#",";"]
)

print(response["choices"][0]["message"]["content"].strip(" \n"))

### Postgres SQL tables, with their properties:
#
# Employee(id, name, department_id)
# Department(id, name, address)
# Salary_Payments(id, employee_id, amount, date)
#
### A query to list the names of the departments which employed more than 10 employees in the last 3 months
\ query: SELECT

Department.name 
FROM Department 
INNER JOIN Employee ON Department.id = Employee.department_id 
INNER JOIN Salary_Payments ON Employee.id = Salary_Payments.employee_id 
WHERE Salary_Payments.date >= NOW() - INTERVAL '3 months' 
GROUP BY Department.id 
HAVING COUNT(Employee.id) > 10


In [14]:
prompt="Can you explain what does this code do?\n#\n# ###\n\
   Code:\n\
   SELECT d.name FROM Department d JOIN Employee e ON d.id = e.department_id WHERE e.id IN (SELECT employee_id FROM Salary_Payments WHERE date > now() - interval '3 months') GROUP BY d.name HAVING COUNT(*) > 10\n#\n#\
   Answer:\n# "

# prompt="이 코드가 무엇을 수행하는지 설명해줘?\n#\n# ###\n\
#    Code:\n\
#    SELECT d.name FROM Department d JOIN Employee e ON d.id = e.department_id WHERE e.id IN (SELECT employee_id FROM Salary_Payments WHERE date > now() - interval '3 months') GROUP BY d.name HAVING COUNT(*) > 10\n#\n#\
#    Answer:\n# "
   
response = openai.ChatCompletion.create(
  engine=os.getenv('DEPLOYMENT_NAME'),
  messages=[
    {"role": "user", "content": prompt},
  ],
   max_tokens=250,
  stop=["#",";"])

print(response["choices"][0]["message"]["content"].strip(" \n"))

This code selects the names of all departments that have more than 10 employees who have received salary payments in the last 3 months. It does this by joining the Department and Employee tables on their respective ids, and filtering the result to only include employees who have received salary payments in the last 3 months (using a subquery and the "date" field in the Salary_Payments table). Then, it groups the result by department name and counts how many employees in each department have received salary payments in the last 3 months. Departments with more than 10 such employees are included in the final result.
